In this example we will search for different AOIs and dates, organized in a csv file. However, notice that the main function to search the Planet images is access_planet_images.planet_images, which requires as input:

- AOI
- start_date
- end_date
- cloud cover
- intersection area (in percentage) between the Planet image and the AOI
- API_KEY
- Output folder name

In [ ]:
import json
import pandas as pd
import geopandas as gpd
import access_planet_images
from shapely.ops import transform

In [ ]:
API_KEY = 'INCLUDE_HERE_YOUR_PLANET_API_KEY'
area_coverage = 20

csv_dataset = r"csv_example.csv"
output_path = "out_json_example.json"
dataset_df = pd.read_csv(csv_dataset)
dataset_df['date'] = pd.to_datetime(dataset_df['date'], format='%Y%m%d')
dataset_df['next_day'] = dataset_df['date'] + pd.Timedelta(days=1)
dataset_df['date'] = dataset_df['date'].dt.strftime('%Y-%m-%d')
dataset_df['next_day'] = dataset_df['next_day'].dt.strftime('%Y-%m-%d')


In [ ]:
merged_filtered_dates = {}
merged_grouped_images = {}

for i in range(0,len(dataset_df)):

    tile = dataset_df.iloc[i]['tile']
    start_date = dataset_df.iloc[i]['date']
    end_date = dataset_df.iloc[i]['next_day']
    gpkg_file_path = dataset_df.iloc[i]['gpkg_path']

    gpkg_file = gpd.read_file(gpkg_file_path).to_crs(4326)

    geojson_str = gpkg_file.to_json()
    geojson_obj = json.loads(geojson_str) # Dictionary

    grouped_images = {}
    filtered_dates = {}

    print(f"Running {tile}, {start_date}")

    for k in range(len(geojson_obj['features'])):
        feature = geojson_obj['features'][k]

        try:
            filtered_dates_aux, grouped_images_aux = access_planet_images.planet_images(feature["geometry"], start_date, end_date, 30, area_coverage, API_KEY, tile)
        except: continue

        if not grouped_images_aux:
            print(f"No images found for {tile}, {start_date}")
            continue

        if not filtered_dates_aux:
            filtered_dates_aux = grouped_images_aux

        key_g = list(grouped_images_aux.keys())[0]
        key_f = list(filtered_dates_aux.keys())[0]

        if key_g not in grouped_images:
            grouped_images[key_g] = []
        if key_f not in filtered_dates:
            filtered_dates[key_f] = []

        grouped_images[key_g].extend(grouped_images_aux[key_g])
        filtered_dates[key_f].extend(filtered_dates_aux[key_f])

    merged_grouped_images.update(grouped_images)
    merged_filtered_dates.update(filtered_dates)

dates_list = list(merged_filtered_dates.keys())

with open(output_path, 'w') as f:
    json.dump(merged_grouped_images, f)


Once we searched for all available images, we can download them.

In [ ]:
from src.download_planet import DownloadPlanet

image_out_path = "path/to/you/folder"

planet = DownloadPlanet(image_out_path, API_KEY)

with open(output_path, "r") as f:
    data = json.load(f)

for date_key, scenes in data.items():
    for scene in scenes:
        planet.process_scene(scene)